In [9]:
with open('test-input.txt') as f:
    test_full_text = f.read()

with open('input.txt') as f:
    prod_full_text = f.read()
  
# full_text = prod_full_text
full_text = prod_full_text


import re 
numbers_re = re.compile(r'\d+')

def parse_numbers_list(numbers_list_text):
    return [int(n) for n in numbers_re.findall(numbers_list_text)]

sections = full_text.split('\n\n')
seeds = parse_numbers_list(sections[0].split(': ')[1])

def calculate_greedily(): 
  def generate_dict(destination_range_start, source_range_start, range_length):
      return {source_range_start + i: destination_range_start + i for i in range(range_length)}

  maps = {}

  working_translation = seeds.copy()

  for section in sections[1:]:
      headline, *ranges = section.split('\n')
      headline = headline.replace(' map', '')
      from_item, to_item = headline.split('-to-')
      # print(headline)
      # print(ranges)
      section_dict = {}
      for r in ranges:
          range_dict = generate_dict(*parse_numbers_list(r))
          section_dict.update(range_dict)
      # print(section_dict)
      maps[headline] = {
          'from': from_item,
          'to': to_item,
          'dict': section_dict
      }
      for i in range(len(working_translation)):
          working_translation[i] = section_dict.get(working_translation[i], working_translation[i])
      # destination_range_start, source_range_start, range_length = parse_range(section)
      # seeds = [seeds[i] for i in generate_dict(destination_range_start, source_range_start, range_length).values()]
      
  print('the lowest location number that corresponds to any of the initial seed numbers is:', min(working_translation))



In [10]:
working_translation = seeds.copy()


def binary_search(map_ranges, lookup_key, search_value):
      if len(map_ranges) == 1:
            return map_ranges[0]
      else:
            mid = len(map_ranges) // 2
            if search_value < map_ranges[mid][lookup_key]:
                  return binary_search(map_ranges[:mid], lookup_key, search_value)
            else:
                  return binary_search(map_ranges[mid:], lookup_key, search_value)

def translate_sorted(to_translate_values, ranges):
      map_ranges = []
      for r in ranges:
            range_dist_start, range_source_start, range_length = parse_numbers_list(r)
            map_ranges.append((range_dist_start, range_source_start, range_length))
      map_ranges.sort(key=lambda x: x[1])

      translated_values = []
      for val in to_translate_values:
            likely_range = binary_search(map_ranges, 1, val)
            # print(f'val: {val}, likely_range: {likely_range}')
            if val < (likely_range[1] + likely_range[2]) and val >= likely_range[1]:
                  # print('in range')
                  translated_values.append(likely_range[0] + val - likely_range[1])
            else:
                  # print('out of range')
                  translated_values.append(val)
      return translated_values

for section in sections[1:]:
      headline, *ranges = section.split('\n')
      headline = headline.replace(' map', '')
      from_item, to_item = headline.split('-to-')
      print(headline)
      # print(ranges)
      working_translation = translate_sorted(working_translation, ranges)

print(min(working_translation))

      

seed-to-soil:
soil-to-fertilizer:
fertilizer-to-water:
water-to-light:
light-to-temperature:
temperature-to-humidity:
humidity-to-location:
199602917


In [11]:
# working_translation = []
# for i in range(0, len(seeds), 2):
#       working_translation.extend([seeds[i] + k for k in range(seeds[i + 1])])

# len(working_translation)

# for section in sections[1:]:
#       headline, *ranges = section.split('\n')
#       headline = headline.replace(' map', '')
#       from_item, to_item = headline.split('-to-')
#       print(headline)
#       # print(ranges)
#       working_translation = translate_sorted(working_translation, ranges)

# print(min(working_translation))

In [12]:
from pprint import pprint
"""
We want to identify the smallest location id that we have seeds for. 
"""

maps = []


def parse_ranges(ranges):
    for r in ranges:
        destination_start, source_start, length = parse_numbers_list(
            r
        )
        yield {
            "destination_start": destination_start,
            "source_start": source_start,
            "length": length,
            "destination_end": destination_start + length - 1,
            "source_end": source_start + length - 1,
            "destination_difference": destination_start - source_start,
        }


for section in sections[1:]:
    headline, *ranges = section.split("\n")
    headline = headline.replace(" map", "")
    from_item, to_item = headline.split("-to-")
    # print(headline)
    # print(ranges)
    # print(section_dict)
    # print(list(parse_ranges(ranges)))
    maps.append(
        {
            "from": from_item,
            "to": to_item,
            "ranges": sorted(
                list(parse_ranges(ranges)), key=lambda x: x["destination_start"]
            ),
        }
    )

pprint(maps)
seed_ranges = [[seeds[s], seeds[s] + seeds[s + 1]] for s in range(0, len(seeds), 2)]

def is_in_range(target_range_start: int, target_range_end: int, value: int) -> bool:
    return target_range_start <= value <= target_range_end
    
def apply_map_function(value, destination_start, source_start):
    return value + (destination_start - source_start)

for i, seed_range in enumerate(seed_ranges):
    print(seed_range)
    for m in maps:
        for r in m["ranges"]:
            if is_in_range(r['destination_start'], r['destination_end'], seed_range[0]):
                if is_in_range(r['source_start'], r['source_end'], seed_range[1]):
                    print('source range is fully contained in map range')
                    seed_ranges[i][0] = apply_map_function(seed_range[0], r['destination_start'], r['source_start'])
                    seed_ranges[i][1] = apply_map_function(seed_range[1], r['destination_start'], r['source_start'])
                else:
                    print('source start is contained in map range but source end is not', seed_range, r)
            
            
                

[{'from': 'seed',
  'ranges': [{'destination_difference': -1731434235,
              'destination_end': 73663476,
              'destination_start': 0,
              'length': 73663477,
              'source_end': 1805097711,
              'source_start': 1731434235},
             {'destination_difference': -533542796,
              'destination_end': 144190399,
              'destination_start': 73663477,
              'length': 70526923,
              'source_end': 677733195,
              'source_start': 607206273},
             {'destination_difference': -1023077343,
              'destination_end': 148592688,
              'destination_start': 144190400,
              'length': 4402289,
              'source_end': 1171670031,
              'source_start': 1167267743},
             {'destination_difference': -1234333319,
              'destination_end': 220408858,
              'destination_start': 148592689,
              'length': 71816170,
              'source_end': 1454742177,

In [13]:
from typing import List


class Range:
    def __init__(self, start: int, end: int):
        if start > end:
            raise ValueError(f"start ({start}) must be less than or equal to end ({end})")
        self.start = start
        self.end = end
        self.length = end - start + 1

    def __repr__(self):
        return f"Range({self.start}, {self.end})"

    def __eq__(self, other):
        return self.start == other.start and self.end == other.end

    def __ne__(self, other):
        return not self.__eq__(other)

    def __contains__(self, value: int):
        return self.start <= value <= self.end

    def __add__(self, other):
        if not self.has_overlap(other):
            raise ValueError("ranges do not overlap")
        return Range(min(self.start, other.start), max(self.end, other.end))

    def __gt__(self, other):
        return self.start > other.start

    def __lt__(self, other):
        return self.start < other.start

    def __ge__(self, other):
        return self.start >= other.start

    def __le__(self, other):
        return self.start <= other.start

    def get_between(self, other):
        if self.has_overlap(other):
            return None
        return Range(max(self.start, other.start), min(self.end, other.end))

    def has_overlap(self, other):
        # two ranges can overlap
        ## - self starts before other and ends after other starts
        if self.start <= other.start <= self.end:
            return True
        ## - other starts before self and ends after self starts
        if other.start <= self.start <= other.end:
            return True
        ## - self starts before other and ends after other ends
        if self.start <= other.start and self.end >= other.end:
            return True
        ## - other starts before self and ends after self ends
        if other.start <= self.start and other.end >= self.end:
            return True
        ## - self and other are the same
        if self == other:
            return True
        return False

    def get_overlap(self, other):
        if not self.has_overlap(other):
            return None
        return Range(max(self.start, other.start), min(self.end, other.end))


class MappingFunction:
    def __init__(self, source_start=None, length=None, destination_start=None):
        self.scoped = source_start is not None and length is not None
        self.source_range = (
            Range(source_start, source_start + length - 1) if self.scoped else None
        )
        self.length = length
        self.destination_range = (
            Range(destination_start, destination_start + length - 1)
            if self.scoped
            else None
        )
        self.destination_offset = destination_start - source_start if self.scoped else 0

    def __repr__(self):
        if not self.scoped:
            return "Default Mapping Function"
        return f"MappingFunction(source range: {self.source_range}, destination range: {self.destination_range}, destination offset: {self.destination_offset}"

    def __eq__(self, other):
        return (
            self.source_start == other.source_start
            and self.length == other.length
            and self.destination_start == other.destination_start
        )

    def __ne__(self, other):
        return not self.__eq__(other)

    def apply(self, value: int):
        return value + self.destination_offset

    def apply_range(self, range: Range):
        if not self.scoped:
            return range
        return Range(self.apply(range.start), self.apply(range.end))

    def is_applicable(self, value: int):
        if not self.scoped:
            return True
        return value in self.source_range

    def has_overlap(self, range: Range):
        if not self.scoped:
            return True
        return self.source_range.has_overlap(range)


Default_Mapping_Function = MappingFunction()


def make_scoped_default_mapping_function(range: Range):
    return MappingFunction(range.start, range.length, range.start)


class Map:
    def __init__(self, name: str, from_item: str, to_item: str, ranges: List[str]):
        self.name = name
        self.from_item = from_item
        self.to_item = to_item
        self.mapping_functions = []
        for r in ranges:
            destination_start, source_start, length = parse_numbers_list(r)
            self.mapping_functions.append(
                MappingFunction(source_start, length, destination_start)
            )

    def __repr__(self):
        return f"{self.name} Map"

    def get_applicable_mapping_function(self, value):
        for mf in self.mapping_functions:
            if mf.is_applicable(value):
                return mf
        print(
            "no applicable mapping function found for value", value, "in map", self.name
        )
        return Default_Mapping_Function

    def get_applicable_mapping_functions(self, range: Range):
        return [mf for mf in self.mapping_functions if mf.has_overlap(range)]


class SourceRange:
    def __init__(
        self,
        entity: str,
        source_start: int,
        source_end: int,
        parent: "SourceRange" = None,
    ):
        self.entity = entity
        self.range = Range(source_start, source_end)
        self.parent = parent
        self.mapped_children = []

    def __repr__(self):
        return f"{self.entity} Source{self.range}"

    def apply_map(self, map: Map):
        mapping_functions = map.get_applicable_mapping_functions(self.range)
        if len(mapping_functions) == 0:
            self.mapped_children = [
                SourceRange(map.to_item, self.range.start, self.range.end, self)
            ]
            return self.mapped_children

        mapped_ranges = []
        current_range_start = self.range.start
        current_range_end = self.range.end
        while True:
            # starting from the beginning of the range, apply the first mapping function that applies
            applicable_mapping_function = next(
                (mf for mf in mapping_functions if mf.is_applicable(current_range_start)),
                None,
            )
            if applicable_mapping_function is None:
                # if no mapping function applies for the start, find the first mapping function that applies within the range of the source range. Use the start of that range to scope the mapping of the first portion of the source range
                remaining_mapping_functions = [mf for mf in mapping_functions if mf.has_overlap(Range(current_range_start, current_range_end))]
                if len(remaining_mapping_functions) > 0:
                    next_applicable_mapping_function = sorted(
                        remaining_mapping_functions, key=lambda mf: mf.source_range.start
                    )[0]
                
                    current_range_end = (
                        next_applicable_mapping_function.source_range.start - 1
                    )
                applicable_mapping_function = make_scoped_default_mapping_function(
                    Range(current_range_start, current_range_end)
                )
            current_range_end = min(
                applicable_mapping_function.source_range.end, self.range.end
            )
            applied_range = applicable_mapping_function.apply_range(
                        Range(current_range_start, current_range_end)
                    )
            mapped_ranges.append(
                SourceRange(
                    map.to_item,
                    applied_range.start,
                    applied_range.end,
                    self,
                )
            )
            # repeat with the remaining portion of the source range until the end of the source range is reached
            if current_range_end == self.range.end:
                break
            else:
                current_range_start = current_range_end + 1
                current_range_end = self.range.end

        self.mapped_children = mapped_ranges
        return self.mapped_children


maps = []

for section in sections[1:]:
    headline, *ranges = section.split("\n")
    headline = headline.replace(" map:", "").strip()
    from_item, to_item = headline.split("-to-")
    # print(headline)
    # print(ranges)
    # print(section_dict)
    # print(list(parse_ranges(ranges)))
    maps.append(Map(headline, from_item, to_item, ranges))

source_ranges = [
    SourceRange("seed", seeds[s], seeds[s] + seeds[s + 1] - 1)
    for s in range(0, len(seeds), 2)
]
# source_ranges = [
#     SourceRange("seed", seeds[s], seeds[s]) for s in range(0, len(seeds), 1)
# ]
for map in maps:
    next_source_ranges = []
    for source_range in source_ranges:
        next_source_ranges.extend(source_range.apply_map(map))

    source_ranges = next_source_ranges

print(min([sr.range.start for sr in source_ranges]))

2254686
